In [7]:
# Required imports
import os
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [8]:
cpi_df = pd.read_csv(
    Path("resources/foodPrice.csv")
)

cpi_df.head()

,Cereals-and-bakery-products,"Meats,-poultry,-fish,-and-eggs",Dairy-and-related-products,Fruits-and-vegetables,Other-food-at-home,Date
0,79.7,90.4,87.3,76.5,83.0,1980-01-01
1,80.6,89.7,87.8,76.0,84.3,1980-02-01
2,81.2,90.3,88.1,77.4,85.5,1980-03-01
3,82.4,89.3,88.9,80.2,86.4,1980-04-01
4,83.2,88.0,90.5,82.1,87.3,1980-05-01


In [14]:
for year in np.arange(2005,2022+1):
    spendDf = pd.read_csv(Path(f"resources/spending_{year}.csv"),index_col = 'Item')

In [15]:
spendDf

,Chicago,Detroit,Minneapolis-St.Paul,St. Louis,New York,Philadelphia,Boston,"Washington, D.C.",Baltimore,Atlanta,...,Tampa,Los Angeles,San Francisco,San Diego,Seattle,Phoenix,Denver,Honolulu,Anchorage,Year
Item,,,,,,,,,,,,,,,,,,,,,
"Meats, poultry, fish, and eggs",1269,1184,1100,1170,1534,1269,1198,1391,1398,1260,...,1019,1311,1789,1210,1190,982,1239,1731,1576,2022
Dairy products,492,510,571,534,664,558,577,613,554,477,...,522,502,740,563,620,466,530,590,725,2022
Fruits and vegetables,1115,1055,1241,1045,1460,1237,1414,1261,1329,1204,...,906,1217,1958,1416,1368,870,1103,1703,1931,2022
Other food at home,2035,2051,2317,2252,2163,2252,2461,2488,2102,2096,...,1768,1937,2815,2334,2678,1892,1861,2699,3183,2022
Food away from home,3741,2968,3507,3817,4004,4366,4312,4768,4488,3322,...,3156,4495,4781,4205,3934,3517,4538,5146,3611,2022
Alcoholic beverages,907,655,1098,789,788,871,1100,929,692,600,...,799,720,998,900,1269,815,898,559,831,2022
Housing,27212,21746,27591,22255,31213,28453,31679,33285,26047,24914,...,23686,28300,41367,31853,33090,24843,30893,27799,25141,2022
Apparel and services,2534,1744,1823,1772,2699,3597,1807,1965,2296,2442,...,1516,2126,2199,2549,2398,1834,1781,1411,2083,2022
Transportation,9569,12566,10018,12262,10326,11247,11697,12162,14457,12354,...,11940,13112,12991,12619,14481,11736,12994,10103,13745,2022
